# Data Preparation for UT3000 Nightly Mood Prediction

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import sys
sys.path.append('../')

from src.visualization import visualize

import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.cm import ScalarMappable

from scipy import stats
from sklearn.linear_model import LinearRegression

<a id='toc'></a>

# Table of Contents

1. [Data Import](#data_import)
2. [Pre-Processing](#pre_processing)
3. [Inspection](#inspection)
4. [Analysis](#analysis)

---

<a id='data_import'></a>

[Back to ToC](#toc)
# Data Import
We have two datasets to import:

## EMAs

In [67]:
ema_1000 = pd.read_csv("/Volumes/HEF_Dissertation_Research/utx000/data/raw/ut1000/beiwe/content.csv",index_col=0,parse_dates=["date","datetime"],infer_datetime_format=True)
ema_1000.rename({"answer":"answer_content"},axis="columns",inplace=True)
for file in os.listdir("/Volumes/HEF_Dissertation_Research/utx000/data/raw/ut1000/beiwe/"):
    temp = pd.read_csv(f"/Volumes/HEF_Dissertation_Research/utx000/data/raw/ut1000/beiwe/{file}",index_col=0,parse_dates=["date","datetime"],infer_datetime_format=True)
    mood = file.split(".")[0]
    if mood != "content":
        temp.rename({"answer":f"answer_{mood}"},axis="columns",inplace=True)
        if mood in ["sleep","refreshed","restful"]:
            temp.sort_values(["pid","date"],inplace=True)
            temp.drop(["datetime"],axis="columns",inplace=True)
            temp.drop_duplicates(subset=["date","pid"],keep="first",inplace=True)
            ema_1000 = ema_1000.merge(temp,on=["date","pid"])
        else:
            ema_1000 = ema_1000.merge(temp,on=["date","datetime","pid"])
        print(mood)
        print(len(ema_1000))

energy
15832
lonely
15754
refreshed
13429
restful
13386
sad
13351
sleep
13351
stress
13316


In [68]:
ema_1000.sort_values(["pid","date"])[:20]

,date,datetime,answer_content,pid,answer_energy,answer_lonely,answer_refreshed,answer_restful,answer_sad,answer_sleep,answer_stress
0,2018-10-17,2018-10-17 14:15:51,2,11i3mr4n,2,1,2,2,0,7.0,1
1,2018-10-19,2018-10-19 19:06:20,2,11i3mr4n,2,1,1,1,0,7.0,1
2,2018-10-23,2018-10-23 13:01:16,2,11i3mr4n,2,0,1,1,0,5.0,1
3,2018-10-27,2018-10-27 23:29:35,2,11i3mr4n,2,0,1,2,0,9.0,0
4,2018-10-11,2018-10-11 21:41:02,1,11jbbat5,2,0,2,3,2,5.0,3
5,2018-10-12,2018-10-12 23:26:33,1,11jbbat5,1,1,0,1,2,4.0,3
6,2018-10-14,2018-10-14 23:36:56,2,11jbbat5,3,0,1,2,1,12.0,3
7,2018-10-15,2018-10-15 22:38:02,2,11jbbat5,1,0,1,1,0,NaN,1
8,2018-10-16,2018-10-16 22:36:20,2,11jbbat5,2,0,2,2,1,3.0,2
9,2018-10-17,2018-10-18 09:49:38,1,11jbbat5,0,0,2,2,3,7.0,3


[Back to Data Import](#data_import)

---

<a id='pre_processing'></a>

[Back to ToC](#toc)
# Pre-Processing

[Back to Pre-Processing](#pre_processing)

---

<a id='inspection'></a>

[Back to ToC](#toc)
# Inspection
Various functions and initial looks at the raw and pre-processed data.

[Back to Inspection](#inspection)

---

<a id='analysis'></a>

[Back to ToC](#toc)
# Analysis
The following is the general analysis that follows:

[Back to Analysis](#analysis)

---

<a id='time_dependence'></a>